Импортирую библиотеки, тут использовал только pandas и numpy

In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.precision", 3)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.max_rows", 25)

Загружаю датасеты

In [2]:
df_chech = pd.read_csv('data/row/cheques_rows.csv', sep=';')
df_prod = pd.read_csv('data/row/product_df.csv', sep=";")

Объединяю датасеты

In [3]:
df = pd.merge(df_chech, df_prod, how='left', left_on='product_id', right_on='product_id')

Перевожу временную фичу в формат datetime

In [4]:
df['sale_date_id'] =  pd.to_datetime(df['sale_date_id'], format='%Y%m%d')

Добавляю фичу со статусом продажи - по акции или нет, если размер скидки  больше 0.05 от общей цены

In [5]:
df['sale'] = np.where(df['discount_prc'] >  (df['sum_discounted'] + df['discount_prc'] )* 0.05, 'Sale', 'Reg')

Проверка на наличие неодназначных продаж. Таких продаж, которые проходили в один день, в одной подгруппе товаров, в разных чеках, но с разными условиями - по акции и нет

In [6]:
df['status_g'] = np.where(
    (df['sale_date_id'] == df['sale_date_id']) &
    (df['product_sub_group_id'] == df['product_sub_group_id'])&
    (df['cheque_id'] != df['cheque_id'])  &
    (df['sale'] != df['sale'])
    ,'Bad', 'Good'
)

Неодназначных продаж не обнаруженно

In [7]:
df.loc[df['status_g'] == 'Bad']

,cheque_id,sale_date_id,contact_id,shop_id,product_id,quantity,sum_discounted,discount_prc,product_sub_group_id,sale,status_g


На всякий случай, делаю такую же проверку для всех товаров

In [8]:
df['status_p'] = np.where(
    (df['sale_date_id'] == df['sale_date_id']) &
    (df['product_id'] == df['product_id'])&
    (df['cheque_id'] != df['cheque_id'])  &
    (df['sale'] != df['sale'])
    ,'Bad', 'Good'
)

In [9]:
df.loc[df['status_p'] == 'Bad']

,cheque_id,sale_date_id,contact_id,shop_id,product_id,quantity,sum_discounted,discount_prc,product_sub_group_id,sale,status_g,status_p


### По результатам вычислений, неодназначных продаж не обнаруженно

Подсчитываю. Сколько чеков было по товарным группам было по акции и по регулярной цене 

In [10]:
df_checks_by_status = df.groupby(by=['sale', 'product_sub_group_id'], as_index = False).agg({'cheque_id' : pd.Series.nunique})

Подсчитываю, скольно дней товар продавался по акции и нет, и в каком магазине 

In [59]:
df_days = df.groupby(by=['sale', 'product_sub_group_id' ,  'shop_id'], as_index = False).agg({'sale_date_id': pd.Series.nunique })


In [63]:
df_days_with_out_shop = df.groupby(by=['sale', 'product_sub_group_id' ], as_index = False).agg({'sale_date_id': pd.Series.nunique })

Основные показатели для продаж по регулярной цене для товаров

In [28]:
df_reg = df.loc[df['sale'] == 'Reg'].groupby('product_id').agg({
    'cheque_id' : 'count',
    'quantity' : 'sum', 
    'sum_discounted': 'sum'
}).rename({
    'cheque_id': 'cheque_count_reg', 
    'quantity': 'quantity_reg', 
    'sum_discounted': 'sum_reg'}, axis='columns')

Можно было не делать два разных датасета 

In [30]:
df_sales = df.loc[df['sale'] == 'Sale'].groupby('product_id').agg({
    'cheque_id' : 'count',
    'quantity' : 'sum', 
    'sum_discounted': 'sum'
}).rename({
    'cheque_id': 'cheque_count_sales', 
    'quantity': 'quantity_sales', 
    'sum_discounted': 'sum_sales'}, axis='columns')

In [31]:
df_sum_items = pd.concat([df_reg, df_sales], axis=1)

Основные показатели для товарных подгрупп 

/ Можно было не создавать два дата сете

In [18]:
df_reg_group = df.loc[df['sale'] == 'Reg'].groupby('product_sub_group_id', as_index = False).agg({
    'cheque_id' : 'count',
    'quantity' : 'sum', 
    'sum_discounted': 'sum'
}).rename({
    'cheque_id': 'cheque_count_reg', 
    'quantity': 'quantity_reg', 
    'sum_discounted': 'sum_reg'}, axis='columns')

In [19]:
df_sale_group = df.loc[df['sale'] == 'Sale'].groupby('product_sub_group_id', as_index = False).agg({
    'cheque_id' : 'count',
    'quantity' : 'sum', 
    'sum_discounted': 'sum'
}).rename({
    'cheque_id': 'cheque_count_reg', 
    'quantity': 'quantity_reg', 
    'sum_discounted': 'sum_reg'}, axis='columns')

Собираю датасет по показателяем

In [45]:
df_sum_group = pd.concat([df_reg_group, df_sale_group], axis=1)

Функция подсчета коэффицента роста 

In [46]:
def coefficent(x, y):
    return (y/x)*100

## Подсчитываю основные показатели для продуктов

In [47]:
df_sum_items['c_check'] = coefficent(df_sum_items['cheque_count_reg'],df_sum_items['cheque_count_sales'])
df_sum_items['c_qua'] = coefficent(df_sum_items['quantity_reg'],df_sum_items['quantity_sales'])
df_sum_items['c_sum'] = coefficent(df_sum_items['sum_reg'],df_sum_items['sum_sales'])

## Подсчитываю основные показатели темпов роста для групп

### Дальше чищу от товаров, которые продавались только по акции или только по регулярной цене. Я их удаляю, но мне кажется, что можно подставить 0

In [48]:
df_sum_items.dropna(inplace=True)

In [49]:
df_sum = pd.merge(df_sum_items, df_prod, how='left', left_on='product_id', right_on='product_id')

In [50]:
df_sum.head()

,product_id,cheque_count_reg,quantity_reg,sum_reg,cheque_count_sales,quantity_sales,sum_sales,c_check,c_qua,c_sum,product_sub_group_id
0,0,3961.0,6448.556,4.281e+05,1087.0,1440.893,6.405e+04,27.443,22.344,14.960,0
1,101,276.0,298.000,3.019e+04,31.0,32.000,2.548e+03,11.232,10.738,8.439,0
2,107,185.0,218.000,5.640e+04,57.0,71.000,1.079e+04,30.811,32.569,19.125,1
3,108,475.0,532.000,8.965e+04,193.0,240.000,2.440e+04,40.632,45.113,27.216,1
4,109,11068.0,22229.000,2.256e+06,17543.0,17751.000,1.758e+06,158.502,79.855,77.956,1


Подсчитываю показатели 

In [51]:
df_sum['r_check'] = (df_sum['cheque_count_sales'] * df_sum['c_check']) 

In [52]:
def r_coef(df_sub):
    return df_sub['r_check'].sum() / df_sub['quantity_sales'].sum()

In [53]:
df_sum_group = df_sum.groupby('product_sub_group_id').apply(r_coef)

In [54]:
result = pd.DataFrame({'product_sub_group_coef_sales':df_sum_group.values}, index=df_sum_group.index)

## Получается датасет по группам товаров с коэффицентом темпа роста 

In [55]:
result

,product_sub_group_coef_sales
product_sub_group_id,
0,20.489
1,155.872
2,155.534
3,201.618
4,97.191
...,...
1417,254.605
1418,30.030
1419,34.705


Данные по дням продаж по магазинам, где и в каком проводились акции и по каким товарным группам

In [65]:
result_2 = pd.merge(result, df_days_with_out_shop.loc[df_days_with_out_shop['sale'] == 'Sale'], how='left', left_on='product_sub_group_id', right_on='product_sub_group_id')

### Тут лучше было бы сделать датасет по всем магазинам, где в каких магазинах были акции где нет 

In [68]:
count_days = df.agg({'sale_date_id' : 'nunique'})
result_2['reg_date_count'] =  count_days.item() - result_2['sale_date_id']


In [70]:
del result_2['sale']

In [75]:
result_3 = pd.merge(result_2, df_checks_by_status.loc[df_checks_by_status['sale'] == 'Sale'],  how='left', left_on='product_sub_group_id', right_on='product_sub_group_id')

In [77]:
result_4 = pd.merge(result_3, df_checks_by_status.loc[df_checks_by_status['sale'] == 'Reg'],  how='left', left_on='product_sub_group_id', right_on='product_sub_group_id')

In [83]:
result_4.rename(columns={'cheque_id_x': 'check_by_sale', 'cheque_id_y' : 'check_by_reg'})

,product_sub_group_id,product_sub_group_coef_sales,sale_date_id,reg_date_count,check_by_sale,check_by_reg
0,0,20.489,26,2,1118,4238
1,1,155.872,28,0,17796,11716
2,2,155.534,28,0,4580,4849
3,3,201.618,28,0,7465,11688
4,4,97.191,28,0,1567,1479
...,...,...,...,...,...,...
827,1417,254.605,28,0,13062,11702
828,1418,30.030,9,19,18,247
829,1419,34.705,4,24,5,21
830,1421,6.816,13,15,48,497


# Финальный результат 

In [82]:
result_4

,product_sub_group_id,product_sub_group_coef_sales,sale_date_id,reg_date_count,cheque_id_x,cheque_id_y
0,0,20.489,26,2,1118,4238
1,1,155.872,28,0,17796,11716
2,2,155.534,28,0,4580,4849
3,3,201.618,28,0,7465,11688
4,4,97.191,28,0,1567,1479
...,...,...,...,...,...,...
827,1417,254.605,28,0,13062,11702
828,1418,30.030,9,19,18,247
829,1419,34.705,4,24,5,21
830,1421,6.816,13,15,48,497


In [85]:
result_4.to_csv('data/final/final.csv')

# Показатели

In [88]:
df_mean_cheque = df.groupby(by=['sale']).agg({'sum_discounted' : 'sum', 'cheque_id' : 'count' })
df_mean_cheque['mean_cheque'] = df_mean_cheque['sum_discounted'] / df_mean_cheque['cheque_id']

### Средний чек

In [90]:
df_mean_cheque

,sum_discounted,cheque_id,mean_cheque
sale,,,
Reg,7.467e+08,8292019,90.055
Sale,6.304e+08,7246258,86.995


### Покупки по клиентам, в каком магазине покупали, со скидкой или без

In [91]:
df.groupby(by=['sale', 'contact_id', 'shop_id']).agg({'sum_discounted' : 'sum'})

sum_discounted
sale contact_id shop_id                
Reg  1          1               1096.37
                34                78.81
     2          2              15044.25
                32               906.81
     3          3               1732.66
...                                 ...
Sale 267540     65                94.99
     267543     45               272.90
     267545     75               319.90
     267546     32                39.90
     267549     9                159.90

[1030444 rows x 1 columns]

### Частота посещений по клиентам

In [95]:
df_freq = df.groupby(by=['contact_id'], as_index=False).agg({'sale_date_id': "nunique" })
df_freq

,contact_id,sale_date_id
0,1,5
1,2,25
2,3,9
3,4,7
4,5,11
...,...,...
267544,267545,1
267545,267546,1
267546,267547,1
267547,267548,1


### Частота посещений в среднем

In [103]:
df_freq_mean = df_freq.agg({'contact_id':'count', 'sale_date_id' : 'sum'})
df_freq_mean['mean'] = df_freq_mean['contact_id'] / df_freq_mean['sale_date_id']
df_freq_mean

contact_id      2.675e+05
sale_date_id    1.865e+06
mean            1.434e-01
dtype: float64